---
title: Model to Correct Homophone Misue
author: Marion Bauman
---


For our natural language processing model, we will be building a tool that corrects homophone misuse.

Our model will compute the likelihood that a given word is correct, and if it is not, it will suggest a replacement word. This will be tested on a corpus of data including some intentional homophone misuse.

In [28]:
from transformers import BertForMaskedLM, BertTokenizer, AdamW, get_linear_schedule_with_warmup

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
# Chat GPT Prompt: Give me a list of lists in python of 100 sets of homophones
homophones_list = [
    ["ate", "eight"],
    ["bare", "bear"],
    ["brake", "break"],
    ["capital", "capitol"],
    ["cell", "sell"],
    ["cite", "site", "sight"],
    ["complement", "compliment"],
    ["desert", "dessert"],
    ["die", "dye"],
    ["flour", "flower"],
    ["hear", "here"],
    ["hour", "our"],
    ["knight", "night"],
    ["know", "no"],
    ["mail", "male"],
    ["meat", "meet"],
    ["morning", "mourning"],
    ["one", "won"],
    ["pair", "pear"],
    ["peace", "piece"],
    ["principal", "principle"],
    ["rain", "reign", "rein"],
    ["right", "write"],
    ["sea", "see"],
    ["serial", "cereal"],
    ["sole", "soul"],
    ["stationary", "stationery"],
    ["tail", "tale"],
    ["threw", "through"],
    ["to", "too", "two"],
    ["weather", "whether"],
    ["week", "weak"],
    ["wear", "where"],
    ["which", "witch"],
    ["your", "you're"],
    ["allowed", "aloud"],
    ["board", "bored"],
    ["brake", "break"],
    ["capital", "capitol"],
    ["compliment", "complement"],
    ["desert", "dessert"],
    ["dual", "duel"],
    ["fair", "fare"],
    ["genre", "jinja"],
    ["hare", "hair"],
    ["here", "hear"],
    ["hoard", "horde"],
    ["loan", "lone"],
    ["pail", "pale"],
    ["peak", "peek", "pique"],
    ["profit", "prophet"],
    ["role", "roll"],
    ["root", "route"],
    ["sail", "sale"],
    ["scene", "seen"],
    ["serial", "cereal"],
    ["so", "sow"],
    ["stare", "stair"],
    ["steal", "steel"],
    ["their", "there", "they're"],
    ["throne", "thrown"],
    ["vain", "vein", "vane"],
    ["weak", "week"],
    ["wood", "would"],
    ["yew", "you"],
    ["bridal", "bridle"],
    ["cereal", "serial"],
    ["chord", "cord"],
    ["compliment", "complement"],
    ["dew", "due"],
    ["foul", "fowl"],
    ["grate", "great"],
    ["groan", "grown"],
    ["heal", "heel"],
    ["him", "hymn"],
    ["lay", "lie"],
    ["main", "mane"],
    ["marry", "merry"],
    ["mite", "might"],
    ["moose", "mousse"],
    ["mourn", "morn"],
    ["peace", "piece"],
    ["plum", "plumb"],
    ["pour", "pore"],
    ["rap", "wrap"],
    ["scene", "seen"],
    ["scent", "cent", "sent"],
    ["serial", "cereal"],
    ["shear", "sheer"],
    ["soar", "sore"],
    ["sow", "sew"],
    ["stake", "steak"],
    ["tide", "tied"],
    ["toe", "tow"],
    ["there", "their", "they're"],
    ["waist", "waste"],
    ["week", "weak"],
    ["write", "right", "rite"],
]

In [4]:
import pandas as pd

gutenberg_homophone_data = pd.read_csv('../data/gutenberg-homophone-errors.csv')

In [62]:
test_sentence = """He doesn't no how to code."""

In [63]:
tokenized_sentence = tokenizer.tokenize(test_sentence)
print(tokenized_sentence)

['He', 'doesn', "'", 't', 'no', 'how', 'to', 'code', '.']


In [64]:
# flatten the list of lists
homophones_list_flat = [item for sublist in homophones_list for item in sublist]

In [76]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", "bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


PipelineException: No mask_token ([MASK]) found on the input

In [95]:
import torch
testy = test_sentence.split()
outs = []
for ts in range(0, len(testy)):
    ts_full = testy.copy()[:ts+1]
    print(ts_full)
    if ts_full[ts] in homophones_list_flat:
        print(ts_full[ts])
        ts_full[ts] = '[MASK]'
        ts_full = ' '.join(ts_full)
        print(ts_full)
        print(mask_filler(ts_full, top_k=3))
        outs.append(mask_filler(ts_full, top_k=5))

['He']
['He', "doesn't"]
['He', "doesn't", 'no']
no
He doesn't [MASK]
[{'score': 0.9814466238021851, 'token': 119, 'token_str': '.', 'sequence': "He doesn't."}, {'score': 0.009747138246893883, 'token': 132, 'token_str': ';', 'sequence': "He doesn't ;"}, {'score': 0.00484459986910224, 'token': 106, 'token_str': '!', 'sequence': "He doesn't!"}]


['He', "doesn't", 'no', 'how']
['He', "doesn't", 'no', 'how', 'to']
to
He doesn't no how [MASK]
[{'score': 0.7354617714881897, 'token': 119, 'token_str': '.', 'sequence': "He doesn't no how."}, {'score': 0.1854487657546997, 'token': 136, 'token_str': '?', 'sequence': "He doesn't no how?"}, {'score': 0.055269792675971985, 'token': 106, 'token_str': '!', 'sequence': "He doesn't no how!"}]
['He', "doesn't", 'no', 'how', 'to', 'code.']


In [93]:
outs

[[{'score': 0.9814466238021851,
   'token': 119,
   'token_str': '.',
   'sequence': "He doesn't."},
  {'score': 0.009747138246893883,
   'token': 132,
   'token_str': ';',
   'sequence': "He doesn't ;"},
  {'score': 0.00484459986910224,
   'token': 106,
   'token_str': '!',
   'sequence': "He doesn't!"},
  {'score': 0.003609249135479331,
   'token': 136,
   'token_str': '?',
   'sequence': "He doesn't?"},
  {'score': 0.0001153292105300352,
   'token': 1232,
   'token_str': '...',
   'sequence': "He doesn't..."}],
 [{'score': 0.7354617714881897,
   'token': 119,
   'token_str': '.',
   'sequence': "He doesn't no how."},
  {'score': 0.1854487657546997,
   'token': 136,
   'token_str': '?',
   'sequence': "He doesn't no how?"},
  {'score': 0.055269792675971985,
   'token': 106,
   'token_str': '!',
   'sequence': "He doesn't no how!"},
  {'score': 0.01971513405442238,
   'token': 132,
   'token_str': ';',
   'sequence': "He doesn't no how ;"},
  {'score': 0.0010626193834468722,
   'token

In [ ]:
all_homophones = [
    ['accessary', 'accessory'],
    ['ad', 'add'],
    ['ail', 'ale'],
    ['air', 'heir'],
    ['aisle', "I'll", 'isle'],
    ['all', 'awl'],
    ['allowed', 'aloud'],
    ['altar', 'alter'],
    ['arc', 'ark'],
    ['ant', 'aunt'],
    ['ate', 'eight'],
    ['auger', 'augur'],
    ['auk', 'orc'],
    ['aural', 'oral'],
    ['away', 'aweigh'],
    ['aw', 'awe'],
    ['ore', 'oar', 'or'],
    ['axel', 'axle'],
    ['aye', 'eye', 'I'],
    ['bail', 'bale'],
    ['bait', 'bate'],
    ['baize', 'bays'],
    ['bald', 'bawled'],
    ['ball', 'bawl'],
    ['band', 'banned'],
    ['bard', 'barred'],
    ['bare', 'bear'],
    ['bark', 'barque'],
    ['baron', 'barren'],
    ['base', 'bass'],
    ['based', 'baste'],
    ['bazaar', 'bizarre'],
    ['be', 'bee'],
    ['bay', 'bey'],
    ['beach', 'beech'],
    ['bean', 'been'],
    ['beat', 'beet'],
    ['beau', 'bow'],
    ['beer', 'bier'],
    ['bel', 'bell', 'belle'],
    ['berry', 'bury'],
    ['berth', 'birth'],
    ['bight', 'bite', 'byte'],
    ['billed', 'build'],
    ['bitten', 'bittern'],
    ['blew', 'blue'],
    ['bloc', 'block', 'bloque'],
    ['boar', 'bore'],
    ['board', 'bored'],
    ['boarder', 'border'],
    ['bold', 'bowled'],
    ['boos', 'booze'],
    ['born', 'borne'],
    ['bough', 'bow'],
    ['boy', 'buoy'],
    ['brae', 'bray'],
    ['braid', 'brayed'],
    ['braise', 'brays', 'braze'],
    ['brake', 'break'],
    ['bread', 'bred'],
    ['brews', 'bruise'],
    ['bridal', 'bridle'],
    ['broach', 'brooch'],
    ['bur', 'burr', 'brr'],
    ['but', 'butt'],
    ['buy', 'by', 'bye'],
    ['buyer', 'byre'],
    ['calendar', 'calender'],
    ['call', 'caul'],
    ['canvas', 'canvass'],
    ['cast', 'caste'],
    ['caster', 'castor'],
    ['caught', 'court'],
    ['caw', 'core', 'corps'],
    ['cede', 'seed'],
    ['ceiling', 'sealing'],
    ['cell', 'sell'],
    ['censer', 'censor', 'sensor'],
    ['cent', 'scent', 'sent'],
    ['cereal', 'serial'],
    ['cheap', 'cheep'],
    ['check', 'cheque'],
    ['choir', 'quire'],
    ['chord', 'cord'],
    ['cite', 'sight', 'site'],
    ['clack', 'claque'],
    ['clew', 'clue'],
    ['climb', 'clime'],
    ['close', 'cloze'],
    ['coal', 'kohl'],
    ['coarse', 'course'],
    ['coign', 'coin'],
    ['colonel', 'kernel'],
    ['complacent', 'complaisant'],
    ['complement', 'compliment'],
    ['coo', 'coup'],
    ['cops', 'copse'],
    ['council', 'counsel'],
    ['cousin', 'cozen'],
    ['creak', 'creek'],
    ['crews', 'cruise'],
    ['cue', 'kyu', 'queue'],
    ['curb', 'kerb'],
    ['currant', 'current'],
    ['cymbol', 'symbol'],
    ['dam', 'damn'],
    ['days', 'daze'],
    ['dear', 'deer'],
    ['descent', 'dissent'],
    ['desert', 'dessert'],
    ['deviser', 'divisor'],
    ['dew', 'due'],
    ['die', 'dye'],
    ['discreet', 'discrete'],
    ['doe', 'doh', 'dough'],
    ['done', 'dun'],
    ['douse', 'dowse'],
    ['draft', 'draught'],
    ['dual', 'duel'],
    ['earn', 'urn'],
    ['eery', 'eyrie'],
    ['ewe', 'yew', 'you'],
    ['faint', 'feint'],
    ['fah', 'far'],
    ['fair', 'fare'],
    ['fairy', 'ferry'],
    ['fate', 'fete'],
    ['farther', 'father'],
    ['faun', 'fawn'],
    ['faze', 'phase'],
    ['fay', 'fey'],
    ['feat', 'feet'],
    ['ferrule', 'ferule'],
    ['few', 'phew'],
    ['fie', 'phi'],
    ['file', 'phial'],
    ['find', 'fined'],
    ['fir', 'fur'],
    ['fizz', 'phiz'],
    ['flair', 'flare'],
    ['flaw', 'floor'],
    ['flea', 'flee'],
    ['flex', 'flecks'],
    ['flew', 'flu', 'flue'],
    ['floe', 'flow'],
    ['flour', 'flower'],
    ['for', 'fore', 'four'],
    ['foreword', 'forward'],
    ['fort', 'fought'],
    ['forth', 'fourth'],
    ['foul', 'fowl'],
    ['franc', 'frank'],
    ['freeze', 'frieze'],
    ['friar', 'fryer'],
    ['furs', 'furze'],
    ['gait', 'gate'],
    ['galipot', 'gallipot'],
    ['gamble', 'gambol'],
    ['gallop', 'galop'],
    ['gays', 'gaze'],
    ['genes', 'jeans'],
    ['gild', 'guild'],
    ['gilt', 'guilt'],
    ['giro', 'gyro'],
    ['gnaw', 'nor'],
    ['gneiss', 'nice'],
    ['gorilla', 'guerilla'],
    ['grate', 'great'],
    ['greave', 'grieve'],
    ['greys', 'graze'],
    ['grisly', 'grizzly'],
    ['groan', 'grown'],
    ['guessed', 'guest'],
    ['hail', 'hale'],
    ['hair', 'hare'],
    ['hall', 'haul'],
    ['hangar', 'hanger'],
    ['hart', 'heart'],
    ['haw', 'hoar', 'whore'],
    ['hay', 'hey'],
    ['heal', 'heel', "he'll"],
    ['here', 'hear'],
    ['heard', 'herd'],
    ["he'd", 'heed'],
    ['heroin', 'heroine'],
    ['hew', 'hue'],
    ['hi', 'high'],
    ['higher', 'hire'],
    ['him', 'hymn'],
    ['ho', 'hoe'],
    ['hoard', 'horde'],
    ['hoarse', 'horse'],
    ['holey', 'holy', 'wholly'],
    ['hour', 'our'],
    ['idle', 'idol'],
    ['in', 'inn'],
    ['indict', 'indite'],
    ["it's", 'its'],
    ['jewel', 'joule', 'juul'],
    ['key', 'quay'],
    ['knave', 'nave'],
    ['knead', 'need'],
    ['knew', 'new'],
    ['knight', 'night'],
    ['knit', 'nit'],
    ['knob', 'nob'],
    ['knock', 'nock'],
    ['knot', 'not'],
    ['know', 'no'],
    ['knows', 'nose'],
    ['laager', 'lager'],
    ['lac', 'lack'],
    ['lade', 'laid'],
    ['lain', 'lane'],
    ['lam', 'lamb'],
    ['laps', 'lapse'],
    ['larva', 'lava'],
    ['lase', 'laze'],
    ['law', 'lore'],
    ['lay', 'ley'],
    ['lea', 'lee'],
    ['leach', 'leech'],
    ['lead', 'led'],
    ['leak', 'leek'],
    ['lean', 'lien'],
    ['lessen', 'lesson'],
    ['levee', 'levy'],
    ['liar', 'lyre'],
    ['licence', 'license'],
    ['licker', 'liquor'],
    ['lie', 'lye'],
    ['lieu', 'loo'],
    ['links', 'lynx'],
    ['lo', 'low'],
    ['load', 'lode'],
    ['loan', 'lone'],
    ['locks', 'lox'],
    ['loop', 'loupe'],
    ['loot', 'lute'],
    ['made', 'maid'],
    ['mail', 'male'],
    ['main', 'mane'],
    ['maize', 'maze'],
    ['mall', 'maul'],
    ['manna', 'manner'],
    ['mantel', 'mantle'],
    ['mare', 'mayor'],
    ['mark', 'marque'],
    ['marshal', 'martial'],
    ['marten', 'martin'],
    ['mask', 'masque'],
    ['maw', 'more'],
    ['me', 'mi'],
    ['mean', 'mien'],
    ['meat', 'meet', 'mete'],
    ['medal', 'meddle'],
    ['metal', 'mettle'],
    ['meter', 'metre'],
    ['might', 'mite'],
    ['miner', 'minor', 'mynah'],
    ['mind', 'mined'],
    ['missed', 'mist'],
    ['moat', 'mote'],
    ['mode', 'mowed'],
    ['moor', 'more'],
    ['moose', 'mousse'],
    ['morning', 'mourning'],
    ['muscle', 'mussel'],
    ['naval', 'navel'],
    ['nay', 'neigh'],
    ['nigh', 'nye'],
    ['none', 'nun'],
    ['od', 'odd'],
    ['ode', 'owed'],
    ['oh', 'owe'],
    ['one', 'won'],
    ['packed', 'pact'],
    ['packs', 'pax'],
    ['pail', 'pale'],
    ['pain', 'pane'],
    ['pair', 'pare', 'pear'],
    ['palate', 'palette', 'pallet'],
    ['pascal', 'paschal'],
    ['paten', 'patten', 'pattern'],
    ['pause', 'paws', 'pores', 'pours'],
    ['peace', 'piece'],
    ['peak', 'peek', 'pique', 'peke'],
    ['pea', 'pee'],
    ['peal', 'peel'],
    ['pearl', 'purl'],
    ['pedal', 'peddle'],
    ['peer', 'pier'],
    ['pi', 'pie'],
    ['pica', 'pika'],
    ['place', 'plaice'],
    ['plain', 'plane'],
    ['pleas', 'please'],
    ['pole', 'poll'],
    ['plum', 'plumb'],
    ['poof', 'pouffe'],
    ['practice', 'practise'],
    ['praise', 'prays', 'preys'],
    ['principal', 'principle'],
    ['profit', 'prophet'],
    ['quarts', 'quartz'],
    ['quean', 'queen'],
    ['rain', 'reign', 'rein'],
    ['raise', 'rays', 'raze'],
    ['rap', 'wrap'],
    ['raw', 'roar'],
    ['read', 'reed'],
    ['read', 'red'],
    ['real', 'reel'],
    ['reek', 'wreak'],
    ['rest', 'wrest'],
    ['retch', 'wretch'],
    ['review', 'revue'],
    ['rheum', 'room'],
    ['right', 'rite', 'wright', 'write'],
    ['ring', 'wring'],
    ['road', 'rode'],
    ['roe', 'row'],
    ['role', 'roll'],
    ['roo', 'roux', 'rue'],
    ['rood', 'rude'],
    ['root', 'route'],
    ['rose', 'rows'],
    ['rota', 'rotor'],
    ['rote', 'wrote'],
    ['rough', 'ruff'],
    ['rouse', 'rows'],
    ['rung', 'wrung'],
    ['rye', 'wry'],
    ['saver', 'savour'],
    ['scull', 'skull'],
    ['spade', 'spayed'],
    ['sale', 'sail'],
    ['sane', 'seine'],
    ['satire', 'satyr'],
    ['sauce', 'source'],
    ['saw', 'soar', 'sore'],
    ['scene', 'seen'],
    ['sea', 'see'],
    ['seam', 'seem'],
    ['sear', 'seer', 'sere'],
    ['seas', 'sees', 'seize'],
    ['shake', 'sheikh'],
    ['sew', 'so', 'sow'],
    ['shear', 'sheer'],
    ['shoe', 'shoo'],
    ['sic', 'sick'],
    ['side', 'sighed'],
    ['sign', 'sine'],
    ['sink', 'synch'],
    ['slay', 'sleigh'],
    ['sloe', 'slow'],
    ['sole', 'soul'],
    ['some', 'sum'],
    ['son', 'sun'],
    ['sort', 'sought'],
    ['spa', 'spar'],
    ['staid', 'stayed'],
    ['stair', 'stare'],
    ['stake', 'steak'],
    ['stalk', 'stork'],
    ['stationary', 'stationery'],
    ['steal', 'steel'],
    ['stile', 'style'],
    ['storey', 'story'],
    ['straight', 'strait'],
    ['sweet', 'suite'],
    ['swat', 'swot'],
    ['tacks', 'tax'],
    ['tale', 'tail'],
    ['talk', 'torque'],
    ['tare', 'tear'],
    ['taught', 'taut', 'tort'],
    ['te', 'tea', 'tee', 't', 'ti'],
    ['team', 'teem'],
    ['tear', 'tier'],
    ['teas', 'tease'],
    ['terce', 'terse'],
    ['tern', 'turn'],
    ['there', 'their', "they're"],
    ['threw', 'through', 'thru'],
    ['throes', 'throws'],
    ['throne', 'thrown'],
    ['thyme', 'time'],
    ['tic', 'tick'],
    ['tide', 'tied'],
    ['tire', 'tyre'],
    ['to', 'too', 'two'],
    ['toad', 'toed', 'towed'],
    ['told', 'tolled'],
    ['tole', 'toll'],
    ['ton', 'tun'],
    ['tor', 'tore'],
    ['tough', 'tuff'],
    ['troop', 'troupe'],
    ['tuba', 'tuber'],
    ['vain', 'vane', 'vein'],
    ['vale', 'veil'],
    ['vial', 'vile'],
    ['vice', 'vise'],
    ['wade', 'weighed'],
    ['weak', 'week'],
    ['we', 'wee', 'whee'],
    ['way', 'weigh', 'whey'],
    ['wax', 'whacks'],
    ['wart', 'wort'],
    ['watt', 'what'],
    ['warn', 'worn'],
    ['ware', 'wear', 'where'],
    ['war', 'wore'],
    ['wall', 'waul'],
    ['waive', 'wave'],
    ['wait', 'weight'],
    ['wail', 'wale', 'whale'],
    ['wain', 'wane'],
    ["we'd", 'weed'],
    ['weal', "we'll", 'wheel'],
    ['wean', 'ween'],
    ['weather', 'whether'],
    ['weaver', 'weever'],
    ['weir', "we're"],
    ['were', 'whirr'],
    ['wet', 'whet'],
    ['wheald', 'wheeled'],
    ['which', 'witch'],
    ['whig', 'wig'],
    ['while', 'wile'],
    ['whine', 'wine'],
    ['whirl', 'whorl'],
    ['whirled', 'world'],
    ['whit', 'wit'],
    ['white', 'wight'],
    ["who's", 'whose'],
    ['woe', 'whoa'],
    ['wood', 'would'],
    ['yaw', 'yore', 'your', "you're"],
    ['yoke', 'yolk'],
    ["you'll", 'yule']
]